# Project: ICD-AIS conversion using Deep Learning

This converts ICD codes to AIS using an SMT translation file.

## Setup

In [1]:
import numpy as np
import pandas as pd
import sqlite3
import random
import math

## Parameters


In [2]:
icd_obs_file ="../Data/test_icd_pre_I9_A05.csv"
ais_map_codes_file = "../Data/AIS08_mapped_codes.csv"
smt_map_file = "../Results/SMT_ICD_AIS_map.csv"
ais_codes_file = "../Data/AIS08_codes.csv"

## Load data

In [3]:
# load ICD codes
icd_codes = pd.read_csv(icd_obs_file, header=None, names=["ICD9CODE"])

In [9]:
# load ICD-AIS map
smt_map = pd.read_csv(smt_map_file, header=0).rename(columns={'icd_code':'icd9_code'})

In [10]:
smt_map.head(5)

,icd9_code,ais_code,prob
0,D004.2,241699,0.001267
1,D004.9,856161,0.000054
2,D005.9,251000,0.000023
3,D008.42,854455,0.000034
4,D008.45,545699,0.023131


In [11]:
# load AIS codes
ais_codes = pd.read_csv(ais_codes_file, header=0, encoding='iso-8859-1')

# get predot code
ais_codes['predot'] = ais_codes.code.apply(lambda x: math.floor(x))

# make dictionary of codes
ais_dot = pd.Series(ais_codes.code.values, index=ais_codes.predot).to_dict()
ais_dot[0] = 0.0
ais_dot[-1] = -1.0

## Add severity code

In [12]:
ais_dot[10000]

10000.1

In [13]:
smt_map['ais_code'] = smt_map.apply(lambda x: ais_dot[x.ais_code], axis=1)

In [14]:
smt_map

,icd9_code,ais_code,prob
0,D004.2,241699.1,0.001267
1,D004.9,856161.3,0.000054
2,D005.9,251000.1,0.000023
3,D008.42,854455.2,0.000034
4,D008.45,545699.1,0.023131
...,...,...,...
6112,D999.51,853221.3,0.000023
6113,D999.80,544226.4,0.000081
6114,D999.88,140656.5,0.000021
6115,D999.89,878163.1,0.006730


## Convert ICD codes with tidy format

In [15]:
# split strings into separate columns (wide format)
icd_codes = icd_codes.ICD9CODE.str.split(' ', expand=True)

In [16]:
icd_codes

,0,1,2,3,4,5,6,7,8,9,...,117,118,119,120,121,122,123,124,125,126
0,A79,E880.9,P87.03,P87.44,D850.0,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,A55,E884.9,P-2,D805.2,D805.4,D807.03,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,A70,E885.9,P79.35,P87.03,D820.21,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,A56,E881.0,P88.01,P88.38,D805.4,D823.01,D825.20,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,A40,E986,P54.11,D879.4,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122369,A22,E881.1,P78.13,D812.40,D813.40,D813.43,D850.9,D884.0,None,None,...,None,None,None,None,None,None,None,None,None,None
122370,A66,E885.9,P-2,D852.01,D873.43,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
122371,A83,E888.9,P87.03,P88.38,P93.90,D852.05,D852.25,None,None,None,...,None,None,None,None,None,None,None,None,None,None
122372,A50,E816.0,P34.04,D807.09,D860.0,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [17]:
# convert to long format
icd_codes = pd.DataFrame(icd_codes.stack()).reset_index().rename(columns={'level_0':'key',0:'icd9_code'}).drop(columns=['level_1'])

## Trim to only diagnosis codes

In [18]:
# remove all codes that are not 'D' codes
icd_codes = icd_codes[icd_codes.icd9_code.str.contains('D')].reset_index(drop=True)
#icd_codes = icd_codes[~icd_codes.icd9_code.str.contains('V')].reset_index(drop=True)

In [19]:
# remove leading 'D' 
#icd_codes['icd9_code'] = icd_codes.icd9_code.apply(lambda x: x.lstrip('D'))

In [20]:
len(icd_codes)

440800

In [21]:
icd_codes

,key,icd9_code
0,0,D850.0
1,1,D805.2
2,1,D805.4
3,1,D807.03
4,2,D820.21
...,...,...
440795,122372,D807.09
440796,122372,D860.0
440797,122373,D807.02
440798,122373,D812.31


In [22]:
smt_map

,icd9_code,ais_code,prob
0,D004.2,241699.1,0.001267
1,D004.9,856161.3,0.000054
2,D005.9,251000.1,0.000023
3,D008.42,854455.2,0.000034
4,D008.45,545699.1,0.023131
...,...,...,...
6112,D999.51,853221.3,0.000023
6113,D999.80,544226.4,0.000081
6114,D999.88,140656.5,0.000021
6115,D999.89,878163.1,0.006730


## Map ICD9 to AIS 08

In [23]:
# merge map onto ICD codes
icd_codes = icd_codes.merge(smt_map[['icd9_code','ais_code']], how='left', left_on='icd9_code', right_on='icd9_code')

In [24]:
# sort in ascending order
icd_codes = icd_codes.sort_values(['key','icd9_code'])

In [25]:
# convert codes to numbers
icd_codes['ais_code'] = icd_codes.ais_code.astype(np.float64)

In [26]:
# fill in unmapped codes with '-1'
icd_codes = icd_codes.fillna(-1)

In [27]:
# fill in uspecified codes with 0
icd_codes = icd_codes.replace(-100000, 0)

## Evaluate map

In [28]:
# number of codes that do not map
print("Unmatched", sum(icd_codes.ais_code==-1))
print("% Unmatched", sum(icd_codes.ais_code==-1)/len(icd_codes)*100)

Unmatched 115
% Unmatched 0.026088929219600723


In [29]:
# number of unspecified codes
print("Unspecified", sum(icd_codes.ais_code==0))
print("% Unspecified", sum(icd_codes.ais_code==0)/len(icd_codes)*100)

Unspecified 0
% Unspecified 0.0


In [30]:
# number of codes
len(icd_codes)

440800

In [31]:
# number of patients
len(icd_codes.key.unique())

122374

## Convert to codes list

In [32]:
icd_codes = icd_codes.groupby('key')['ais_code'].apply(list).reset_index(name='ais_code').drop(columns=['key'])

In [33]:
icd_codes.head(5)

,ais_code
0,[161001.1]
1,"[650420.2, 650620.2, 450203.3]"
2,[853151.3]
3,"[650620.2, 854471.2, 852004.2]"
4,[510602.1]


## Store results

In [34]:
icd_codes.to_csv("../Results/test_ais_pred_smt_map.csv", index=False, header=False)